In [6]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [7]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame import Frame

periods = 365

m1 = RandomOHLCV( 
    freq      = '1 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

m5 = RandomOHLCV( 
    freq      = '5 mins', 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
).get_dataframe()



f = Frame('TSLA', run_ta_on_load=True)
f.load_ohlcv(m1)
f.import_data(m5, ['close'], prefix='SPY_')
f.add_ta(ta.MA('SPY_close', 5), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1)

f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='')
# f.add_ta(ta.MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
# f.add_ta(ta.MA('close', 21), {'dash': 'solid', 'color': 'cyan', 'width': 1})
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1)
f.add_ta(ta.MA('close', 100), {'dash': 'solid', 'color': 'cyan', 'width': 3}, row=1)
f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'yellow', 'width': 5}, row=1)
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2)
# f.add_ta(ta.MACD(fast=12, slow=26, signal=9) , 
#          [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
#           {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
#           {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')

f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='HP_hi_10', atrCol='ATR_50', slopeDir='down', slopeToleranceATR=2, projectionPeriod=10),
            {'dash': 'solid', 'color': 'rgb(173, 90, 255)', 'width': 1}, 
            chart_type = 'trendlines')

f.add_ta(ta.SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance') 

f.add_ta(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, limit_zones=2),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons')

# f.setup_chart() 

f.backtest_setup(
    start=-50,  # This will slice the data from 0:400
    end=-1,    # We'll test up to row 450
    save_snapshots=True
)

f.backtest_run(clear_snapshots=True)

Tested rows 49/49 - Date: 2024-01-01 06:03:00 (location 363)


In [8]:
f.backtest_data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_close', 'MA_SP_5',
       'ATR_50', 'MA_cl_50', 'MA_cl_100', 'MA_cl_150', 'MA_vo_10', 'HP_hi_10',
       'LP_lo_10', 'MTREND_DOWN_1', 'MTREND_DOWN_2', 'MTREND_DOWN_3', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'CONS_UPPER_1', 'CONS_LOWER_1', 'CONS_UPPER_2',
       'CONS_LOWER_2', 'MTREND_DOWN_4'],
      dtype='object')

In [11]:
f.run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.1, width=2000, height=1500)
# f.get_snapshot(22, plot=True, width=2000, height=1500)

Viewing snapshot 50/50 - Date: 2024-01-01 06:03:00 (location 363)


In [10]:
f.data# .tail(100)
m5.tail(50)
f.backtest_data.tail(50)

,open,high,low,close,volume,SPY_close,MA_SP_5,ATR_50,MA_cl_50,MA_cl_100,...,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,CONS_UPPER_1,CONS_LOWER_1,CONS_UPPER_2,CONS_LOWER_2,MTREND_DOWN_4
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 05:14:00,92.73,93.14,92.51,92.88,5,100.05,100.050,0.4734,93.7864,94.2645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:15:00,92.73,92.91,92.32,92.47,6,100.21,100.082,0.4732,93.7376,94.2362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:16:00,92.80,93.12,92.68,92.89,5,100.21,100.114,0.4696,93.7020,94.2150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:17:00,92.58,92.66,92.20,92.26,5,100.21,100.146,0.4688,93.6638,94.1905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:18:00,92.59,92.79,92.39,92.72,5,100.21,100.178,0.4692,93.6264,94.1714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:19:00,92.80,92.84,92.34,92.45,6,100.21,100.210,0.4728,93.5764,94.1489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:20:00,92.76,93.02,92.58,92.74,5,100.06,100.180,0.4752,93.5374,94.1292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:21:00,92.47,92.76,92.24,92.49,6,100.06,100.150,0.4798,93.4976,94.1005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 05:22:00,92.68,92.91,92.35,92.36,6,100.06,100.120,0.4786,93.4636,94.0677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
